In [33]:
# Importing the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from collections import defaultdict
from surprise import SVD
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')

1. Import and warehouse data:

• Import all the given datasets and explore shape and size.

In [3]:
df0 = pd.read_csv("phone_user_review_file_1.csv",encoding='iso-8859-1')
df0

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8
...,...,...,...,...,...,...,...,...,...,...,...
374905,/cellphones/huawei-y3/,11/30/2015,ru,ru,Yandex,market.yandex.ru,8.0,10.0,Ð² ÑÐ°Ð»Ð¾Ð½Ðµ ÐÐ¢Ð¡ ÐºÑÐ¿Ð¸Ð» Ð¿Ð¾ Ð°ÐºÑÐ...,NaN,Huawei Y3 U03
374906,/cellphones/huawei-y3/,11/9/2015,ru,ru,Yandex,market.yandex.ru,8.0,10.0,Ð ÑÐµÐ»Ð¾Ð¼ Ð½ÐµÐ¿Ð»Ð¾Ñ Ð¾Ð¹ Ð´ÐµÐ²Ð°Ð¹Ñ Ð±...,NaN,Huawei Y3 U03
374907,/cellphones/huawei-y3/,9/27/2015,ru,ru,Yandex,market.yandex.ru,10.0,10.0,Ð Ð¾Ð±ÑÐµÐ¼ Ð¸ ÑÐµÐ»Ð¾Ð¼ Ð°Ð¿Ð¿Ð°ÑÐ°Ñ Ð½Ð...,NaN,Huawei Y3 U03
374908,/cellphones/huawei-y3/,9/25/2015,ru,ru,Yandex,market.yandex.ru,8.0,10.0,Ð¡Ð¼Ð°ÑÑÑÐ¾Ð½ Ð´Ð»Ñ Ð½ÐµÑÑÐµÐ±Ð¾Ð²Ð°ÑÐµ...,NaN,Huawei Y3 U03


In [4]:
df1 = pd.read_csv("phone_user_review_file_2.csv",encoding='iso-8859-1')
df1

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/leagoo-lead-7/,4/15/2015,en,us,Amazon,amazon.com,2.0,10.0,"The telephone headset is of poor quality , not...",luis,Leagoo Lead7 5.0 Inch HD JDI LTPS Screen 3G Sm...
1,/cellphones/leagoo-lead-7/,5/23/2015,en,gb,Amazon,amazon.co.uk,10.0,10.0,This is my first smartphone so I have nothing ...,Mark Lavin,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...
2,/cellphones/leagoo-lead-7/,4/27/2015,en,gb,Amazon,amazon.co.uk,8.0,10.0,Great phone. Battery life not great but seems ...,tracey,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...
3,/cellphones/leagoo-lead-7/,4/22/2015,en,gb,Amazon,amazon.co.uk,10.0,10.0,Best 90 quid I've ever spent on a smart phone,Reuben Ingram,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...
4,/cellphones/leagoo-lead-7/,4/18/2015,en,gb,Amazon,amazon.co.uk,10.0,10.0,I m happy with this phone.it s very good.thx team,viorel,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...
...,...,...,...,...,...,...,...,...,...,...,...
114920,/cellphones/motorola-moto-g-4-plus/,8/19/2016,en,in,Amazon,amazon.in,2.0,10.0,It's getting problem when both sim working tog...,raj agrawal,"Motorola Moto G Plus, 4th Gen (White, 32 GB)"
114921,/cellphones/motorola-moto-g-4-plus/,8/18/2016,en,in,Amazon,amazon.in,2.0,10.0,Hopeless phone. Not at all user friendly. Woul...,Amazon Customer,"Motorola Moto G Plus, 4th Gen (White, 32 GB)"
114922,/cellphones/motorola-moto-g-4-plus/,8/18/2016,en,in,Amazon,amazon.in,10.0,10.0,Very nice phone.,Amazon Customer,"Motorola Moto G Plus, 4th Gen (White, 32 GB)"
114923,/cellphones/motorola-moto-g-4-plus/,8/18/2016,en,in,Amazon,amazon.in,8.0,10.0,Over all good phone . Give all the specificati...,Amazon Customer,"Motorola Moto G Plus, 4th Gen (White, 32 GB)"


In [5]:
df2 = pd.read_csv("phone_user_review_file_3.csv",encoding='iso-8859-1')
df2

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,11/7/2015,pt,br,Submarino,submarino.com.br,6.0,10.0,"recomendo, eu comprei um, a um ano, e agora co...",herlington tesch,Samsung Smartphone Samsung Galaxy S3 Slim G381...
1,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,10/2/2015,pt,br,Submarino,submarino.com.br,10.0,10.0,Comprei um pouco desconfiada do site e do celu...,Luisa Silva Marieta,Samsung Smartphone Samsung Galaxy S3 Slim G381...
2,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,9/2/2015,pt,br,Submarino,submarino.com.br,10.0,10.0,"Muito bom o produto, obvio que tem versÃµes me...",Cyrus,Samsung Smartphone Samsung Galaxy S3 Slim G381...
3,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,9/2/2015,pt,br,Submarino,submarino.com.br,8.0,10.0,Unica ressalva fica para a camera que poderia ...,Marcela Santa Clara Brito,Samsung Smartphone Samsung Galaxy S3 Slim G381...
4,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,9/1/2015,pt,br,Colombo,colombo.com.br,8.0,10.0,Rapidez e atenÃ§Ã£o na entrega. O aparelho Ã© ...,Claudine Maria Kuhn Walendorff,"Smartphone Samsung Galaxy S3 Slim, Dual Chip, ..."
...,...,...,...,...,...,...,...,...,...,...,...
312956,/cellphones/samsung-galaxy-s-iii/,1/23/2014,de,de,Yopi,yopi.de,10.0,10.0,Ein Disney- Meisterwerk fÃ¼r jung und alt! Zei...,burnson,Samsung Galaxy Express I8730
312957,/cellphones/samsung-galaxy-s-iii/,1/23/2014,de,de,Yopi,yopi.de,NaN,NaN,Nachdem ich mir einen neuen 3D fÃ¤higen Fernse...,Comflag,Samsung Galaxy Express I8730
312958,/cellphones/samsung-galaxy-s-iii/,1/22/2014,ru,ru,Yandex,market.yandex.ru,8.0,10.0,ÐÐ¾ Ð¿Ð¾Ð²Ð¾Ð´Ñ Ð°ÐºÐºÑÐ¼ÑÐ»ÑÑÐ¾ÑÐ°. Ð...,NaN,Samsung Galaxy S III GT-I9300 16Gb
312959,/cellphones/samsung-galaxy-s-iii/,1/22/2014,pt,br,Submarino,submarino.com.br,10.0,10.0,Ã³timo primeira linha acho que todo mundo gost...,magda vanessa ferreira,Samsung Smartphone Samsung Galaxy S III I9300 ...


In [6]:
df3 = pd.read_csv("phone_user_review_file_4.csv",encoding='iso-8859-1')
df3

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-s7262-duos-galaxy-ace/,3/11/2015,en,us,Amazon,amazon.com,2.0,10.0,was not conpatable with my phone as stated. I ...,Frances DeSimone,Samsung Galaxy Star Pro DUOS S7262 Unlocked Ce...
1,/cellphones/samsung-s7262-duos-galaxy-ace/,17/11/2015,en,in,Zopper,zopper.com,10.0,10.0,Decent Functions and Easy to Operate Pros:- Th...,Expert Review,Samsung Galaxy Star Pro S7262 Black
2,/cellphones/samsung-s7262-duos-galaxy-ace/,29/10/2015,en,in,Amazon,amazon.in,4.0,10.0,Not Good Phone such price. Hang too much and v...,Amazon Customer,Samsung Galaxy Star Pro GT-S7262 (Midnight Black)
3,/cellphones/samsung-s7262-duos-galaxy-ace/,29/10/2015,en,in,Amazon,amazon.in,6.0,10.0,not bad for features,Amazon Customer,Samsung Galaxy Star Pro GT-S7262 (Midnight Black)
4,/cellphones/samsung-s7262-duos-galaxy-ace/,29/10/2015,en,in,Amazon,amazon.in,10.0,10.0,Excellent product,NHK,Samsung Galaxy Star Pro GT-S7262 (Midnight Black)
...,...,...,...,...,...,...,...,...,...,...,...
98279,/cellphones/zte-grand-s-flex/,3/1/2016,de,de,Amazon,amazon.de,2.0,10.0,Dieses Handy macht kein Spa??. Die Wlan Verbin...,Anja Jentsch,"ZTE Grand S Flex Smartphone (12,7 cm (5 Zoll) ..."
98280,/cellphones/zte-grand-s-flex/,31/12/2015,de,de,Amazon,amazon.de,4.0,10.0,Das ZTE Grand S Flex begeistert am Anfang durc...,Daniel Winkler,"ZTE Grand S Flex Smartphone (12,7 cm (5 Zoll) ..."
98281,/cellphones/zte-grand-s-flex/,20/12/2015,de,de,Amazon,amazon.de,2.0,10.0,Das Smartphone h??lt in meinem Falle NICHT was...,Amazon Kunde,"ZTE Grand S Flex Smartphone (12,7 cm (5 Zoll) ..."
98282,/cellphones/zte-grand-s-flex/,24/11/2015,de,de,Amazon,amazon.de,10.0,10.0,Dieses Smartphone habe ich letztes Jahr meinem...,Jens D.,"ZTE Grand S Flex Smartphone (12,7 cm (5 Zoll) ..."


In [7]:
df4 = pd.read_csv("phone_user_review_file_5.csv",encoding='iso-8859-1')
df4

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,2.0,10.0,I bought 1 month before. currently speaker is ...,venkatesh,Karbonn K1616
1,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,6.0,10.0,"I just bought one week back, I have Airtel con...",Venkat,Karbonn K1616
2,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,4.0,10.0,one problem in this handset opera is not worki...,krrish,Karbonn K1616
3,/cellphones/karbonn-k1616/,4/25/2014,en,in,Naaptol,naaptol.com,10.0,10.0,here Karbonn comes up with an another excellen...,BRIJESH CHAUHAN,Karbonn K1616 - Black
4,/cellphones/karbonn-k1616/,4/23/2013,en,in,Naaptol,naaptol.com,10.0,10.0,"What a phone, all so on Naaptol my god 23% off...",Suraj CHAUHAN,Karbonn K1616 - Black
...,...,...,...,...,...,...,...,...,...,...,...
350211,/cellphones/sony-ericsson-z710i/,8/7/2006,fr,fr,GraphMobile,graphmobile.com,10.0,10.0,Pour info il est sur amazon.de a 212.99â¬ s'i...,NaN,Sony-Ericsson Z710i
350212,/cellphones/sony-ericsson-z710i/,8/5/2006,fr,fr,GraphMobile,graphmobile.com,9.0,10.0,HabituÃ© Ã samsung sony nous sort 1 jolie cla...,NaN,Sony-Ericsson Z710i
350213,/cellphones/sony-ericsson-z710i/,7/19/2006,fr,fr,GraphMobile,graphmobile.com,10.0,10.0,"Pour les gens qui ne regarde pas Il fait mp3, ...",NaN,Sony-Ericsson Z710i
350214,/cellphones/sony-ericsson-z710i/,7/9/2006,fr,fr,GraphMobile,graphmobile.com,9.0,10.0,C vrai que sans le mp3 c moyen...,NaN,Sony-Ericsson Z710i


In [8]:
df5 = pd.read_csv("phone_user_review_file_6.csv",encoding='iso-8859-1')
df5

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-instinct-sph-m800/,9/16/2011,en,us,Phone Arena,phonearena.com,8.0,10.0,I've had the phone for awhile and it's a prett...,ajabrams95,Samsung Instinct HD
1,/cellphones/samsung-instinct-sph-m800/,2/13/2014,en,us,Amazon,amazon.com,6.0,10.0,to be clear it is not the sellers fault that t...,Stephanie,Samsung SPH M800 Instinct
2,/cellphones/samsung-instinct-sph-m800/,12/30/2011,en,us,Phone Scoop,phonescoop.com,9.0,10.0,Well i love this phone. i have had ton of phon...,snickers,Instinct M800
3,/cellphones/samsung-instinct-sph-m800/,10/18/2008,en,us,HandCellPhone,handcellphone.com,4.0,10.0,I have had my Instinct for several months now ...,A4C,Samsung Instinct
4,/cellphones/samsung-instinct-sph-m800/,9/6/2008,en,us,Reviewed.com,reviewed.com,6.0,10.0,i have had this instinct phone for about two m...,betaBgood,Samsung Instinct
...,...,...,...,...,...,...,...,...,...,...,...
163832,/cellphones/alcatel-ot-club_1187/,5/12/2000,de,de,Ciao,ciao.de,2.0,10.0,Weil mein Onkel bei ALcatel arbeitet habe ich ...,david.paul,Alcatel Club Plus Handy
163833,/cellphones/alcatel-ot-club_1187/,5/11/2000,de,de,Ciao,ciao.de,10.0,10.0,Hy Liebe Leserinnen und Leser!! Ich habe seit ...,Christiane14,Alcatel Club Plus Handy
163834,/cellphones/alcatel-ot-club_1187/,5/4/2000,de,de,Ciao,ciao.de,2.0,10.0,"Jetzt hat wohl Alcatell gedacht ,sie machen wa...",michaelawr,Alcatel Club Plus Handy
163835,/cellphones/alcatel-ot-club_1187/,5/1/2000,de,de,Ciao,ciao.de,8.0,10.0,Ich bin seit 2 Jahren (stolzer) Besitzer eines...,claudia0815,Alcatel Club Plus Handy


• Merge all datasets onto one and explore final shape and size.

In [9]:
df=pd.concat([df0,df1,df2,df3,df4,df5],axis=0)

In [10]:
df

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8
...,...,...,...,...,...,...,...,...,...,...,...
163832,/cellphones/alcatel-ot-club_1187/,5/12/2000,de,de,Ciao,ciao.de,2.0,10.0,Weil mein Onkel bei ALcatel arbeitet habe ich ...,david.paul,Alcatel Club Plus Handy
163833,/cellphones/alcatel-ot-club_1187/,5/11/2000,de,de,Ciao,ciao.de,10.0,10.0,Hy Liebe Leserinnen und Leser!! Ich habe seit ...,Christiane14,Alcatel Club Plus Handy
163834,/cellphones/alcatel-ot-club_1187/,5/4/2000,de,de,Ciao,ciao.de,2.0,10.0,"Jetzt hat wohl Alcatell gedacht ,sie machen wa...",michaelawr,Alcatel Club Plus Handy
163835,/cellphones/alcatel-ot-club_1187/,5/1/2000,de,de,Ciao,ciao.de,8.0,10.0,Ich bin seit 2 Jahren (stolzer) Besitzer eines...,claudia0815,Alcatel Club Plus Handy


In [11]:
#shape of dataframe
df.shape

(1415133, 11)

In [12]:
df.reset_index(inplace=True)

In [13]:
df.drop(columns='index',inplace=True)

In [14]:
df

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8
...,...,...,...,...,...,...,...,...,...,...,...
1415128,/cellphones/alcatel-ot-club_1187/,5/12/2000,de,de,Ciao,ciao.de,2.0,10.0,Weil mein Onkel bei ALcatel arbeitet habe ich ...,david.paul,Alcatel Club Plus Handy
1415129,/cellphones/alcatel-ot-club_1187/,5/11/2000,de,de,Ciao,ciao.de,10.0,10.0,Hy Liebe Leserinnen und Leser!! Ich habe seit ...,Christiane14,Alcatel Club Plus Handy
1415130,/cellphones/alcatel-ot-club_1187/,5/4/2000,de,de,Ciao,ciao.de,2.0,10.0,"Jetzt hat wohl Alcatell gedacht ,sie machen wa...",michaelawr,Alcatel Club Plus Handy
1415131,/cellphones/alcatel-ot-club_1187/,5/1/2000,de,de,Ciao,ciao.de,8.0,10.0,Ich bin seit 2 Jahren (stolzer) Besitzer eines...,claudia0815,Alcatel Club Plus Handy


In [15]:
df.isnull().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        63489
score_max    63489
extract      19361
author       63202
product          1
dtype: int64

In [16]:
#Round of score to the nearest integer
df['score'] = df['score'].round().astype('Int64')

In [17]:
df['score']

0          10
1          10
2           6
3           9
4           4
           ..
1415128     2
1415129    10
1415130     2
1415131     8
1415132     2
Name: score, Length: 1415133, dtype: Int64

In [18]:
#Droping irrelevant features
df.drop(columns=['phone_url','date','lang','country','source','domain','score_max','extract'],inplace=True)

In [19]:
#Impute  Missing Values 
# 1d. Check for missing values. Impute the missing values if there is any. 
# filling the null values in column 'score' and 'score_max' 

df['score'].fillna(df['score'].median(), inplace=True)
# dropping the null values in columns 'author' and 'product'
df = df.dropna()


In [20]:
df.shape

(1351931, 3)

In [21]:
#  Check for duplicate values and remove them if there is any. 
df = df.drop_duplicates()

In [22]:
# Keep only 1000000 data samples. Use random state=612
df = df.sample(n=1000000, random_state=612)

In [23]:
df

,score,author,product
846575,6,carolin reichenbach,"HTC One S Smartphone (10,9 cm (4,3 Zoll) AMOLE..."
401436,6,nande_86_,Huawei Ascend P7
1359243,6,Marcos,Siemens CF110
276855,8,SidneyCrestani,Samsung Smartphone Samsung Galaxy Alpha Desblo...
623030,10,gladys198,LG G Flex D955
...,...,...,...
151707,6,Carlos,LG X Power Dourado Android 6.0 Tela de 5.3 CÃ¢...
405012,8,Rachel T.,LG Electronics G3 black 32GB
669567,10,aze,Wiko Rainbow Lite Smartphone dÃ©bloquÃ© 4G (Ec...
175206,4,Chrissi,Huawei P8 lite Dual-SIM Smartphone (5 Zoll (12...


In [24]:
# Identify the most rated features.
#sorting on products that got highest mean score
df.groupby('product')['score'].mean().sort_values(ascending=False).head() 

product
'Sony Xperia X (F5122) â White â Dual Sim (Google Android 6.0.1, 5 Display, 2 x CORTEX A72 1.8 GHz + 4 x cortex-a53...                       10.0
Samsung Galaxy Core 2 - White                                                                                                                    10.0
Samsung Galaxy Express I437 Unlocked GSM 4G LTE Touchscreen Android Smartphone - Black                                                           10.0
Samsung Galaxy Express 2 4G LTE (G3815) - Smartphone Vodafone dÃ©bloque Android (ecran 4.5", 5 Mp, 8 Go, Dual-Core 1.7 GHz, 1.5 GB RAM), Bleu    10.0
Doro PhoneEasy 622 - MÃ³vil libre (pantalla 2.4", cÃ¡mara 2 Mp, 0.04 GB), negro                                                                  10.0
Name: score, dtype: Float64

In [25]:
df.groupby('author')['score'].mean().sort_values(ascending=False).head() 

author
 efef                              10.0
STabUser                           10.0
SUAVETOCK2012                      10.0
SUAREZ_SUAREZ                      10.0
SUAREZ-DUARTE Guillermo Enrique    10.0
Name: score, dtype: Float64

In [26]:
#Identify the users with most number of reviews. 
(df['author'].value_counts()).head()



Amazon Customer    13870
Cliente Amazon      5251
Client d'Amazon     2226
Amazon Kunde        1657
Anonymous           1565
Name: author, dtype: int64

In [27]:
#ar=list(df['product'].value_counts().apply(lambda x: x>50).reset_index().index)
#dfx=df.iloc[ar,:]
#ay=list(dfx['author'].value_counts().apply(lambda x: x>50).reset_index().index)
#dfy=dfx.iloc[ay,:]

In [28]:
# extracting authors who gave greater than 50 ratings
df1 = pd.DataFrame(columns=['author', 'a_count'])
df1['author']=df['author'].value_counts().index.tolist() 
df1['a_count'] = list(df['author'].value_counts() > 50)
# get names of indexes for which count column value is False
index_names = df1[ df1['a_count'] == False ].index 
# drop these row indexes from dataFrame 
df1.drop(index_names, inplace = True) 
df1

,author,a_count
0,Amazon Customer,True
1,Cliente Amazon,True
2,Client d'Amazon,True
3,Amazon Kunde,True
4,Anonymous,True
...,...,...
707,vincent,True
708,Hugo,True
709,JÃ¼rgen,True
710,zercan74,True


In [29]:
# extracting product that got more than 50 ratings
df2 = pd.DataFrame(columns=['product', 'p_count'])
df2['product']=df['product'].value_counts().index.tolist() 
df2['p_count'] = list(df['product'].value_counts() > 50)
# get names of indexes for which count column value is False
index_names = df2[ df2['p_count'] == False ].index 
# drop these row indexes from dataFrame 
df2.drop(index_names, inplace = True)

In [30]:
# selecting data rows where product is having more than 50 ratings.  
df3 = df[df['product'].isin(df2['product'])] 
df3

,score,author,product
846575,6,carolin reichenbach,"HTC One S Smartphone (10,9 cm (4,3 Zoll) AMOLE..."
401436,6,nande_86_,Huawei Ascend P7
276855,8,SidneyCrestani,Samsung Smartphone Samsung Galaxy Alpha Desblo...
535448,10,Rohit,"Apple iPhone 5s (Gold, 16GB)"
31797,10,nj_jules,Samsung Galaxy S7 SM-G930F 32GB Smartphone (Un...
...,...,...,...
278934,10,Zac,Sim Free Sony E5 Mobile Phone
103653,10,Febs1,Samsung Galaxy S6 64GB (Verizon)
711061,10,ÐÐ»ÑÐ³Ð°,Samsung I9082 Galaxy Grand Duos (ÑÐ¸Ð½Ð¸Ð¹)
1385310,4,enrico6ha,Nokia 5100


In [31]:
# selecting data rows from df3 where author has given more than 50 ratings.
# so that we get the data with products having more than 50 ratings and users who have given more than 50 ratings
df4 = df3[df3['author'].isin(df1['author'])]
df4

,score,author,product
783134,10,Domenico,"Nokia 208 Telefono Cellulare, Micro SIM, UMTS,..."
407982,8,Luca,"LG D855 G3 Smartphone, 16 GB, Nero Metallico [..."
875294,6,Steven,Samsung E1200 Sim Free Mobile Phone - Disconti...
319163,10,Amazon Customer,"Huawei Ascend Y550 Smartphone, 4 GB, Nero"
88193,4,Amazon Customer,"Huawei Nexus 6P unlocked smartphone, 32GB Gold..."
...,...,...,...
546110,10,Frank,Apple iPhone 5S 16GB Space Gray
335388,10,Pablo,Motorola Moto G 16 GB - Smartphone libre Andro...
889623,10,Mac,Nokia C2-05
302159,8,Fernando,"ASUS ZenFone 2 Laser Unlocked Smartphone, 3GB ..."


In [32]:
# Report the shape of the final dataset.
df4.shape

(47589, 3)

In [34]:
#Build a popularity based model and recommend top 5 mobile phones.

In [35]:
#calculating the mean score for a product by grouping it.
df_pop = pd.DataFrame(df.groupby('product')['score'].mean()) 
df_pop


,score
product,
"'Sony Xperia X (F5122) â White â Dual Sim (Google Android 6.0.1, 5 Display, 2 x CORTEX A72 1.8 GHz + 4 x cortex-a53...",10.0
"'Sony Xperia X (F5122) â rosa â Dual Sim (Google Android 6.0.1, 5 Display, 2 x CORTEX A72 1.8 GHz + 4 x cortex-a53...",10.0
"(CUBOT) GT88 5.5"" qHD 1.3GHz MTK6572 2-Core Android 4.2.2 3G Phone 8MP CAM 512MB RAM 4GB ROM",8.0
"(DG300 Versione Aggiornata)5'' DOOGEE VOYAGER2 DG310 Dual Flashlights IPS Screen 3G Smartphone Android 4.4 MTK6582 1.3GHz Quad Core Telefono Cellulare Dual SIM 8G ROM OTG OTA GPS WIFI, BIANCO",7.833333
"(Part 4) Conclusion review of Lenovo S820, my vacation phone",9.0
...,...
ä¸­å¹é«éæ©ãDell Venue æºæ §äºé¨æ² å¯¦æ©åé®®,9.0
å¹¾å¯äºçãéç¶²éå¡ç½è² iPhone 4ï¼,9.0
æ©æç½æ C168i,10.0


In [36]:
# calculating the number of ratings a product got
df_pop['rating_counts'] = pd.DataFrame(df.groupby('product')['score'].count())  

In [37]:
# 3. Recommending the 5 mobile phones based in highest mean score and highest number of ratings the product got. 
df_pop.sort_values(by=['score','rating_counts'], ascending=[False,False]).head()

,score,rating_counts
product,,
Motorola Smartphone Motorola Moto X Desbloqueado Preto Android 4.2.2 CÃ¢mera 10MP e Frontal 2MP MemÃ³ria Interna de 16GB GSM,10.0,147
Motorola Smartphone Motorola Moto G Dual Chip Desbloqueado TIM Android 4.3 Tela 4.5 8GB 3G Wi-Fi CÃ¢mera 5MP - Preto,10.0,141
Nokia Smartphone Nokia Lumia 520 Desbloqueado Oi Preto Windows Phone 8 CÃ¢mera 5MP 3G Wi-Fi MemÃ³ria Interna 8G GPS,10.0,140
Samsung Smartphone Galaxy Win Duos Branco Desbloqueado Dual Chip CÃ¢mera 5MP Processador Quad Core 1.2 Ghz Android 4.1 3G Wi- Fi e MemÃ³ria 8GB,10.0,137
Samsung Smartphone Dual Chip Samsung Galaxy SIII Duos Desbloqueado Claro Azul Android 4.1 3G/Wi-Fi CÃ¢mera 5MP,10.0,134


In [38]:
df

,score,author,product
846575,6,carolin reichenbach,"HTC One S Smartphone (10,9 cm (4,3 Zoll) AMOLE..."
401436,6,nande_86_,Huawei Ascend P7
1359243,6,Marcos,Siemens CF110
276855,8,SidneyCrestani,Samsung Smartphone Samsung Galaxy Alpha Desblo...
623030,10,gladys198,LG G Flex D955
...,...,...,...
151707,6,Carlos,LG X Power Dourado Android 6.0 Tela de 5.3 CÃ¢...
405012,8,Rachel T.,LG Electronics G3 black 32GB
669567,10,aze,Wiko Rainbow Lite Smartphone dÃ©bloquÃ© 4G (Ec...
175206,4,Chrissi,Huawei P8 lite Dual-SIM Smartphone (5 Zoll (12...


In [39]:
#Build a collaborative filtering model using SVD.
# arranging columns in the order of user id,item id and rating to be fed in the svd
columns_titles = ['author','product','score']
df_c = df.reindex(columns=columns_titles)


df_c

,author,product,score
846575,carolin reichenbach,"HTC One S Smartphone (10,9 cm (4,3 Zoll) AMOLE...",6
401436,nande_86_,Huawei Ascend P7,6
1359243,Marcos,Siemens CF110,6
276855,SidneyCrestani,Samsung Smartphone Samsung Galaxy Alpha Desblo...,8
623030,gladys198,LG G Flex D955,10
...,...,...,...
151707,Carlos,LG X Power Dourado Android 6.0 Tela de 5.3 CÃ¢...,6
405012,Rachel T.,LG Electronics G3 black 32GB,8
669567,aze,Wiko Rainbow Lite Smartphone dÃ©bloquÃ© 4G (Ec...,10
175206,Chrissi,Huawei P8 lite Dual-SIM Smartphone (5 Zoll (12...,4


In [40]:
# Keep only 5000 data samples. Use random state=612
df_c5 = df_c.sample(n=5000, random_state=612)
# 4. Build a collaborative filtering model using SVD. 

In [41]:
# 4. Build a collaborative filtering model using SVD. 
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df_c5,reader = reader)
data

In [42]:
#trainset = data.build_full_trainset()
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
#testset = trainset.build_anti_testset()

In [43]:
# Split data to train and test
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=.25,random_state=123)


In [44]:
algo = SVD()           #SVD(n_factors=5,biased=False)
algo.fit(trainset)

In [45]:
predictions = algo.test(testset)

In [46]:

accuracy.rmse(predictions)

RMSE: 2.4956


2.495637804524366

In [47]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    2.5228  2.5195  2.6055  2.5405  2.5310  2.5439  0.0317  
MAE (testset)     1.9678  1.9527  2.0225  1.9775  1.9815  1.9804  0.0233  
Fit time          0.49    0.43    0.45    0.49    0.46    0.46    0.02    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    


{'test_rmse': array([2.52279703, 2.51952882, 2.60549375, 2.54047226, 2.53097267]),
 'test_mae': array([1.9677712 , 1.9526602 , 2.02254035, 1.97751442, 1.98149519]),
 'fit_time': (0.4883594512939453,
  0.4298985004425049,
  0.4489166736602783,
  0.49271082878112793,
  0.4613912105560303),
 'test_time': (0.010774850845336914,
  0.010991096496582031,
  0.009796857833862305,
  0.010775327682495117,
  0.010759830474853516)}

In [48]:
def get_top_n(predictions, n=5):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [49]:
# 8. Try and recommend top 5 products for test users
top_n = get_top_n(predictions, n=5)

In [50]:
top_n

defaultdict(list,
            {'lacatusu cosmin': [('iPhone 5S APPLE IPHONE 5S 32GB SILVER UNLOCKED',
               7.715598781339223)],
             'shpi-lya': [('Sony Ericsson K790i', 8.212414431470606)],
             'vijay gupt': [('Panasonic P51 - White', 8.038133333333333)],
             'MichÃ¨le': [('LG Electronics P710 Optimus L7 II Smartphone (10,9 cm (4,3 Zoll) Touchscreen, 1GHz, Dual-Core, 4GB, 768MB RAM, 8-Megapixel-Kamera, Android 4.1) metallisch-schwarz',
               7.623349482494837)],
             'deralex': [('Motorola Timeport P7389', 8.038133333333333)],
             'yaffa bey': [('3650', 8.038133333333333)],
             'mrflincki': [('Sony Ericsson W980i piano black Handy',
               8.038133333333333)],
             'Afop': [('Microsoft Nokia 6700 classic Handy (UMTS, Bluetooth, 5 MP) illuvial pink',
               8.038133333333333)],
             'andredalv': [('Alcatel One Touch Idol 3 Smartphone, 4.7", Dual SIM, Grigio Scuro [Italia]',
          

In [51]:
#Above are the top 5 predicted items and their ratings for test users.

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

lacatusu cosmin ['iPhone 5S APPLE IPHONE 5S 32GB SILVER UNLOCKED']
shpi-lya ['Sony Ericsson K790i']
vijay gupt ['Panasonic P51 - White']
MichÃ¨le ['LG Electronics P710 Optimus L7 II Smartphone (10,9 cm (4,3 Zoll) Touchscreen, 1GHz, Dual-Core, 4GB, 768MB RAM, 8-Megapixel-Kamera, Android 4.1) metallisch-schwarz']
deralex ['Motorola Timeport P7389']
yaffa bey ['3650']
mrflincki ['Sony Ericsson W980i piano black Handy']
Afop ['Microsoft Nokia 6700 classic Handy (UMTS, Bluetooth, 5 MP) illuvial pink']
andredalv ['Alcatel One Touch Idol 3 Smartphone, 4.7", Dual SIM, Grigio Scuro [Italia]']
Yvan781 ['Sony Mobile Xperia Z1 Smartphone dÃ©bloquÃ© 4G (Ecran: 5 pouces - 16 Go - Android 4.1 Jelly Bean) Noir']
Client d'Amazon ["Doogee Nouveau 5.0'' DOOGEE NOVA Y100x IPS OGS 3G Smartphone DÃ©bloquÃ© Android 5.0 Lollipop MT6582 Quad Core 1.3GHz Double SIM 8GB ROM GPS WIFI (avec Ã\x89tui de protection)- pour Orange, SFR, Bouygues, Virgin, Free ISYS, Lebara, Numericable etc (Blanc)", 'Honor 6 Plus Smart

Russn ['HTC One V']
T ['Xperia Sony Xperia SP SIM-free Android Smartphone - White (discontinued by manufacturer)']
HUH? ['HTC One M9+ Plus 32GB Gold on Silver, 5.2", GSM Unlocked International Model, No Warranty']
stang12 ['Microsoft Nokia Lumia 630 Single-SIM Smartphone (4,5 Zoll (11,4 cm) Touch-Display, 8 GB Speicher, Windows 8.1) gelb']
s3anreilly ['Samsung S7275 Galaxy Ace 3 UK SIM-Free Smartphone - White']
Tatiana Adamova ['elephone P9000 4G Smartphone MTK6755 Octa Core 2,0 GHz 4GB RAM 32GB ROM Android 6.0 5.5" Ultra Ã\x89troit 8+13Ð\x9cP CamÃ©ras Frame en Alliage d`Aluminium Empreintes Digitales Charge Rapide avec KKmoon Support']
Olegat0r ['Samsung S5830 Galaxy Ace (Black)']
Michael H. ['Sony Xperia ZL LTE Smartphone (12,7 cm (5 Zoll) Touchscreen, 1,5GHz, Quad-Core, 2GB RAM, 16GB HDD, 13 Megapixel Kamera, Android 4.1) weiÃ\x9f']
ebr-arr-01 ['Sony Ericsson S312']
Jucan ['Doro Matra - PhoneEasy 605 - T??l??phone portable - GSM - Tri-bande - gris fonc??']
olga ['BlackBerry Curve 93

In [53]:
algo_i = KNNWithMeans(k=10, sim_options={ 'user_based': False})

algo_i.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [54]:
#from surprise.model_selection import cross_validate
#cross_validate(algo_i, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

test_pred=algo_i.test(testset)
print(accuracy.rmse(test_pred))

RMSE: 2.5125
2.512453485068154


In [55]:
from surprise.model_selection import cross_validate
cross_validate(algo_i, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    2.5998  2.6195  2.5645  2.5362  2.5230  2.5686  0.0366  
MAE (testset)     2.0098  2.0216  1.9610  1.9625  1.9631  1.9836  0.0265  
Fit time          0.87    0.53    0.43    0.43    0.45    0.54    0.17    
Test time         0.03    0.02    0.02    0.02    0.03    0.02    0.00    


{'test_rmse': array([2.59975923, 2.61954713, 2.56454752, 2.53620229, 2.52303084]),
 'test_mae': array([2.00976933, 2.02161983, 1.9610476 , 1.96245833, 1.96307775]),
 'fit_time': (0.8659303188323975,
  0.5250492095947266,
  0.43002915382385254,
  0.42806434631347656,
  0.4535372257232666),
 'test_time': (0.026445388793945312,
  0.018604755401611328,
  0.02056884765625,
  0.015673398971557617,
  0.0254666805267334)}

In [56]:
# we can now query for specific predicions
uid = 'Marcos'  # raw user id
iid = 'LG G Flex D955'  # raw item id
# get a prediction for specific users and items.
pred = algo.predict(uid, iid, verbose=True)

# run the trained model against the testset
test_pred = algo_i.test(testset)
#6. Predict score (average rating) for test users
test_pred

user: Marcos     item: LG G Flex D955 r_ui = None   est = 7.72   {'was_impossible': False}


[Prediction(uid='lacatusu cosmin', iid='iPhone 5S APPLE IPHONE 5S 32GB SILVER UNLOCKED', r_ui=10.0, est=10, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid='shpi-lya', iid='Sony Ericsson K790i', r_ui=10.0, est=10, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid='vijay gupt', iid='Panasonic P51 - White', r_ui=10.0, est=10, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid='MichÃ¨le', iid='LG Electronics P710 Optimus L7 II Smartphone (10,9 cm (4,3 Zoll) Touchscreen, 1GHz, Dual-Core, 4GB, 768MB RAM, 8-Megapixel-Kamera, Android 4.1) metallisch-schwarz', r_ui=10.0, est=10, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid='deralex', iid='Motorola Timeport P7389', r_ui=10.0, est=10, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid='yaffa bey', iid='3650', r_ui=9.0, est=9.0, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid='mrflincki', iid='Sony Ericsson W980i piano black Handy', r_ui=4

#Report your findings and inferences :

*Most popular products are	
Motorola Smartphone Motorola Moto X Desbloqueado Preto Android 4.2.2 CÃ¢mera 10MP e Frontal 2MP MemÃ³ria Interna de 16GB GSM,
Motorola Smartphone Motorola Moto G Dual Chip Desbloqueado TIM Android 4.3 Tela 4.5 8GB 3G Wi-Fi CÃ¢mera 5MP - Preto,

Nokia Smartphone Nokia Lumia 520 Desbloqueado Oi Preto Windows Phone 8 CÃ¢mera 5MP 3G Wi-Fi MemÃ³ria Interna 8G GPS,
Samsung Smartphone Galaxy Win Duos Branco Desbloqueado Dual Chip CÃ¢mera 5MP Processador Quad Core 1.2 Ghz Android 4.1 3G Wi- Fi e MemÃ³ria 8GB,
Samsung Smartphone Dual Chip Samsung Galaxy SIII Duos Desbloqueado Claro Azul Android 4.1 3G/Wi-Fi CÃ¢mera 5MP


*products that got highest mean score -Sony Xperia X (F5122)White Dual Sim    

*Amazon Customer are the users with most number of reviews. 



*In what business scenario you should use popularity based Recommendation Systems ?

Populary based Recommendation system recommends products based on popularity and or anything which is in trend. This system is used in many business scenarios such has news businesses, hotel recommendations in a city etc. It is also used in cold start problems where we do not have knowlege about the taste of the user.

*In what business scenario you should use CF based Recommendation Systems ? 

Colleborative filtering recomment product based on the information and taste about the user. Websites like Myntra , Amazon, Nyka, Netflix, Youtube etc use colleborative filltering to recomment their products.

  
*What other possible methods can you think of which can further improve the recommendation for diﬀerent users ?

Apaart from these we can combine the recommendation systems and use hybrid recommendation system, other recommendation system are Content-based, Demographic, Utility based, Knowledge based.


